In [23]:
import psycopg2
from airflow import manager
from tabulate import tabulate


### **Подключение и проверка таблиц**

In [1]:
# Хост: rc1a-p8bp15mmxsfwpbt0.mdb.yandexcloud.net
# Логин: test_user
# Пароль: j2M{CnnFq@
# База: db1

import psycopg2
from psycopg2 import OperationalError

try:
    conn = psycopg2.connect(
        host="rc1a-p8bp15mmxsfwpbt0.mdb.yandexcloud.net", port="6432",
        database="db1",
        user="test_user",
        password="j2M{CnnFq@",
        connect_timeout=10,
    )
    cur = conn.cursor()
    print("Connection successful!")
except OperationalError as e:
    print(f"An error occurred: {e}")




Connection successful!


In [2]:
##вспомогательный метод для вывода данных запросов в виде таблиц
def print_table(rows, desc):
    col_name = [col[0] for col in desc]
    print(tabulate(rows, headers=col_name, tablefmt="fancy_grid"))

In [3]:
## investigation of test.messages table
cur.execute('select * from test.chat_messages limit 15')
print_table(cur.fetchall(), cur.description)

NameError: name 'tabulate' is not defined

In [45]:
##test.managers
##test.rops
cur.execute('select * from test.managers limit 15')
print_table(cur.fetchall(), cur.description)

╒══════════╤═════════════════╤══════════╕
│   mop_id │ name_mop        │   rop_id │
╞══════════╪═════════════════╪══════════╡
│  6645315 │ Гюнель и Илина  │        1 │
├──────────┼─────────────────┼──────────┤
│  6744792 │ Юля и Наташа    │        1 │
├──────────┼─────────────────┼──────────┤
│ 10262505 │ Вика и Марго    │        1 │
├──────────┼─────────────────┼──────────┤
│ 10262513 │ Ира и Варя      │        1 │
├──────────┼─────────────────┼──────────┤
│ 10465254 │ Настя и Даша    │        1 │
├──────────┼─────────────────┼──────────┤
│ 11448610 │ Вика и Катя     │        1 │
├──────────┼─────────────────┼──────────┤
│ 11490270 │ Аня и Ксюша     │        1 │
├──────────┼─────────────────┼──────────┤
│ 11490278 │ Ксюша и Джамиля │        1 │
├──────────┼─────────────────┼──────────┤
│  6780177 │ Даша и Даша     │        2 │
├──────────┼─────────────────┼──────────┤
│  7436430 │ Софа и Алина    │        2 │
├──────────┼─────────────────┼──────────┤
│  7996770 │ Лиза и Ева      │    

In [46]:
cur.execute('select * from test.rops')
print_table(cur.fetchall(), cur.description)

╒══════════╤════════════╕
│   rop_id │ rop_name   │
╞══════════╪════════════╡
│        1 │ Катя РОП   │
├──────────┼────────────┤
│        2 │ Полина РОП │
├──────────┼────────────┤
│        3 │ Эля РОП    │
╘══════════╧════════════╛


In [47]:
cur.execute('select count(*) from test.rops')
print(cur.fetchall())

[(3,)]


### *Запись таблиц в датафреймы*

In [252]:
import pandas as pd
messages = pd.read_sql_query("select * from test.chat_messages", conn)
rops = pd.read_sql_query("select * from test.rops", conn)
manager = pd.read_sql_query("select * from test.managers", conn)

/tmp/ipykernel_57002/2367917083.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  messages = pd.read_sql_query("select * from test.chat_messages", conn)
/tmp/ipykernel_57002/2367917083.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rops = pd.read_sql_query("select * from test.rops", conn)
/tmp/ipykernel_57002/2367917083.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  manager = pd.read_sql_query("select * from test.managers", conn)


In [134]:
messages

,message_id,type,entity_id,created_by,created_at
0,"""01jb7da570sf4f65xdf0ptvv9q""",incoming_chat_message,37556493,0,1730046924
1,"""01jb3t6bc8gvyfnd063nd91ng4""",incoming_chat_message,37549491,0,1729926213
2,"""01jb6gj9ngwk0ybbmg9w90pbqg""",incoming_chat_message,37531455,0,1730016782
3,"""01jb75bkprk0d7hht1g5vy1pm4""",incoming_chat_message,37553371,0,1730038583
4,"""01jb3t5xprbph1c6pym8ervxpm""",incoming_chat_message,37548675,0,1729926199
...,...,...,...,...,...
18924,"""01jb7wekgjash2x5438e4ver43""",outgoing_chat_message,37534717,10262513,1730062798
18925,"""01jb7we9ap70azqhbmqsvxqafh""",outgoing_chat_message,37557233,6780177,1730062787
18926,"""01jb7wdx5x61z0atwjxzff4axn""",outgoing_chat_message,37554115,11448610,1730062775
18927,"""01jb7wd11kcm6500assks44nwk""",outgoing_chat_message,37556525,6780177,1730062746


In [62]:
rops

,rop_id,rop_name
0,1,Катя РОП
1,2,Полина РОП
2,3,Эля РОП


In [63]:
manager

,mop_id,name_mop,rop_id
0,6645315,Гюнель и Илина,1
1,6744792,Юля и Наташа,1
2,10262505,Вика и Марго,1
3,10262513,Ира и Варя,1
4,10465254,Настя и Даша,1
5,11448610,Вика и Катя,1
6,11490270,Аня и Ксюша,1
7,11490278,Ксюша и Джамиля,1
8,6780177,Даша и Даша,2
9,7436430,Софа и Алина,2


## **Первое задание(SQL query)**

Расчёт должен учитывать следующее:

• если в диалоге идут несколько сообщений подряд от клиента или менеджера, то при расчёте времени ответа надо учитывать только первое сообщение из каждого блока;

• менеджеры работают с 09:30 до 00:00, поэтому нерабочее время не должно учитываться в расчёте среднего времени ответа, т.е. если клиент написал в 23:59, а менеджер ответил в 09:30 – время ответа равно одной минуте;

• ответы на сообщения, пришедшие ночью также нужно учитывать.


In [6]:
cur.execute('Rollback')

In [7]:
cur.execute("""
WITH filtered_messages AS (
    SELECT
        cm.entity_id,
        cm.type,
        cm.created_by,
        cm.created_at,
        ROW_NUMBER() OVER (PARTITION BY cm.entity_id, cm.type ORDER BY cm.created_at) AS rn
    FROM test.chat_messages cm
),
first_messages AS (
    SELECT
        fm.entity_id,
        fm.type,
        fm.created_by,
        fm.created_at
    FROM filtered_messages fm
    WHERE fm.rn = 1
),
response_times AS (
    SELECT
        f1.created_by AS manager_id,
        f1.created_at AS manager_time,
        f2.created_at AS client_time,
        CASE
            WHEN f1.created_at >= f2.created_at AND f1.created_at < (f2.created_at + 14 * 3600) THEN f1.created_at - f2.created_at
            WHEN f1.created_at >= (f2.created_at + 14 * 3600) THEN (f2.created_at + 14 * 3600) - f2.created_at
            ELSE 0
        END AS response_time
    FROM first_messages f1
    INNER JOIN first_messages f2 ON f1.entity_id = f2.entity_id AND f1.type = 'outgoing_chat_message' AND f2.type = 'incoming_chat_message'
    WHERE f1.created_by > 0
)
SELECT
    AVG(rt.response_time)/60 AS overall_avg_response_time
FROM response_times rt;
""")

In [8]:
print(cur.fetchall())

[(Decimal('47.4598801113251980'),)]


## **Третье задание(Python, Pandas)**

### *Решение с применением Python, Pandas*

In [253]:
messages

,message_id,type,entity_id,created_by,created_at
0,"""01jb7da570sf4f65xdf0ptvv9q""",incoming_chat_message,37556493,0,1730046924
1,"""01jb3t6bc8gvyfnd063nd91ng4""",incoming_chat_message,37549491,0,1729926213
2,"""01jb6gj9ngwk0ybbmg9w90pbqg""",incoming_chat_message,37531455,0,1730016782
3,"""01jb75bkprk0d7hht1g5vy1pm4""",incoming_chat_message,37553371,0,1730038583
4,"""01jb3t5xprbph1c6pym8ervxpm""",incoming_chat_message,37548675,0,1729926199
...,...,...,...,...,...
18924,"""01jb7wekgjash2x5438e4ver43""",outgoing_chat_message,37534717,10262513,1730062798
18925,"""01jb7we9ap70azqhbmqsvxqafh""",outgoing_chat_message,37557233,6780177,1730062787
18926,"""01jb7wdx5x61z0atwjxzff4axn""",outgoing_chat_message,37554115,11448610,1730062775
18927,"""01jb7wd11kcm6500assks44nwk""",outgoing_chat_message,37556525,6780177,1730062746


In [254]:
## ошибка внутри таблицы клиент сам с собой общается
messages[messages['entity_id']==36541673]

,message_id,type,entity_id,created_by,created_at
18342,"""01jb7t1302g5wj8rbxr2htc838""",incoming_chat_message,36541673,0,1730060258
18344,"""01jb7t17dgj9h4aqy4hdd63ec9""",outgoing_chat_message,36541673,0,1730060262
18655,"""01jb7t0mzh43d4mqyhpfnq3f9q""",outgoing_chat_message,36541673,0,1730060243
18760,"""01jb7t0yknb9dwd08dbcb8abvf""",outgoing_chat_message,36541673,0,1730060253
18762,"""01jb7t0qrw3v1r17e5jxqs87qr""",incoming_chat_message,36541673,0,1730060246


In [212]:
#переводим время в привычный формат из unix time spamp
messages['created_at'] = pd.to_datetime(messages['created_at'], unit='s')

In [213]:
messages['created_by'].unique()

array([       0, 11490270, 11448610,  6645315,  6744792,  7996770,
       11490278,  8558094, 11394134, 10262497, 11394130, 11394126,
       10262485, 10262493, 10262513, 10465690, 10465254,  6780177,
       11550890, 11188802, 11241942,  7417617,  7408305])

In [214]:
#определяем тип сообщения для простоты в понимании
#я выбрала именно столбец type, так как при изменении creted_by будут потеряны идентификаторы менеджеров
messages['type'] = messages['type'].apply(lambda x: "client" if x=='incoming_chat_message' else "manager")
messages

,message_id,type,entity_id,created_by,created_at
0,"""01jb7da570sf4f65xdf0ptvv9q""",client,37556493,0,2024-10-27 16:35:24
1,"""01jb3t6bc8gvyfnd063nd91ng4""",client,37549491,0,2024-10-26 07:03:33
2,"""01jb6gj9ngwk0ybbmg9w90pbqg""",client,37531455,0,2024-10-27 08:13:02
3,"""01jb75bkprk0d7hht1g5vy1pm4""",client,37553371,0,2024-10-27 14:16:23
4,"""01jb3t5xprbph1c6pym8ervxpm""",client,37548675,0,2024-10-26 07:03:19
...,...,...,...,...,...
18924,"""01jb7wekgjash2x5438e4ver43""",manager,37534717,10262513,2024-10-27 20:59:58
18925,"""01jb7we9ap70azqhbmqsvxqafh""",manager,37557233,6780177,2024-10-27 20:59:47
18926,"""01jb7wdx5x61z0atwjxzff4axn""",manager,37554115,11448610,2024-10-27 20:59:35
18927,"""01jb7wd11kcm6500assks44nwk""",manager,37556525,6780177,2024-10-27 20:59:06


In [215]:
## копируем основной датафрейм для фильтрации данных внутри
## так как в условии было сказано учитывать только первый ответ менеджера в блоке, необходимо убрать лишние данные диалога
filter_ms = messages.copy()

## группируем данные по идентификатору сделаки и проставляем номера сообщений
filter_ms['group_number'] = filter_ms.groupby(['entity_id']).cumcount() + 1

filter_ms.sort_values(by=['entity_id', 'created_at'])

## срезаем ненужную часть диалога, то есть сообщения больше 2
filter_ms = filter_ms[filter_ms['group_number'] <= 2 ]
filter_ms = filter_ms.sort_values(by=['entity_id', 'created_at'])
filter_ms

,message_id,type,entity_id,created_by,created_at,group_number
3107,"""01jb4pjb1g22ts7z6arcjv11w6""",client,35810509,0,2024-10-26 15:19:26,1
4102,"""01jb4pncn7gm1re1zdxkzhnp6s""",manager,35810509,10262493,2024-10-26 15:21:05,2
17586,"""01jb7gxwb0sepv9vtekrgj3v6d""",client,35945535,0,2024-10-27 17:38:36,1
17852,"""01jb7h4ad8weqjx1ynrh7s8k25""",manager,35945535,10262493,2024-10-27 17:42:07,2
14805,"""01jb727x90ga4ncv4m705rh0rm""",client,36099337,0,2024-10-27 13:21:56,1
...,...,...,...,...,...,...
18914,"""01jb7w5npw67ghth0ctgqkvrzr""",manager,37557215,6744792,2024-10-27 20:55:05,2
18864,"""01jb7vxxz0djjdc4qjezwhycev""",client,37557225,0,2024-10-27 20:50:52,1
18908,"""01jb7w3wcr7w96me3k4t5y9a3w""",client,37557229,0,2024-10-27 20:54:07,1
18925,"""01jb7we9ap70azqhbmqsvxqafh""",manager,37557233,6780177,2024-10-27 20:59:47,1


In [216]:
# Применяем фильтр для удаления строк без последовательности
# так как в таблице есть сообщения клиентов, так и не получили ответ необходимо провести фильтрацию во избежание проблем при подсчетах
# в коде ниже я создаю новый столбец со смещением -1 для сравнения последовательности
# если ответ от менеджера получен тогда сохраняется последовательность 1 -> 2 в противном случае она прерывается
filter_ms['next'] = filter_ms['group_number'].shift(-1)
filter_ms['next'] = filter_ms['next'].fillna(-1).astype(int)
filter_ms = filter_ms[filter_ms['group_number'] != filter_ms['next']].drop(columns=['next'])

filter_ms

,message_id,type,entity_id,created_by,created_at,group_number
3107,"""01jb4pjb1g22ts7z6arcjv11w6""",client,35810509,0,2024-10-26 15:19:26,1
4102,"""01jb4pncn7gm1re1zdxkzhnp6s""",manager,35810509,10262493,2024-10-26 15:21:05,2
17586,"""01jb7gxwb0sepv9vtekrgj3v6d""",client,35945535,0,2024-10-27 17:38:36,1
17852,"""01jb7h4ad8weqjx1ynrh7s8k25""",manager,35945535,10262493,2024-10-27 17:42:07,2
14805,"""01jb727x90ga4ncv4m705rh0rm""",client,36099337,0,2024-10-27 13:21:56,1
...,...,...,...,...,...,...
18792,"""01jb7v0j8hkm3p8dagt0aae694""",manager,37557187,7996770,2024-10-27 20:34:49,1
18898,"""01jb7wbkf0wfa5j9y4sv6edrvj""",client,37557187,0,2024-10-27 20:58:20,2
18839,"""01jb7vgcc06pd71nvyag5ck7ma""",client,37557215,0,2024-10-27 20:43:28,1
18914,"""01jb7w5npw67ghth0ctgqkvrzr""",manager,37557215,6744792,2024-10-27 20:55:05,2


In [217]:
## дополнительная фильтрация на случай если клиент не получил ответ или менеджер отправил сообщение клиенту без наличия запроса со стороны клиента
filter_ms['next_type'] = filter_ms['type'].shift(-1)
filter_ms['next_type'] = filter_ms['next_type']
filter_ms = filter_ms[filter_ms['type'] != filter_ms['next_type']].drop(columns=['next_type'])
filter_ms

,message_id,type,entity_id,created_by,created_at,group_number
3107,"""01jb4pjb1g22ts7z6arcjv11w6""",client,35810509,0,2024-10-26 15:19:26,1
4102,"""01jb4pncn7gm1re1zdxkzhnp6s""",manager,35810509,10262493,2024-10-26 15:21:05,2
17586,"""01jb7gxwb0sepv9vtekrgj3v6d""",client,35945535,0,2024-10-27 17:38:36,1
17852,"""01jb7h4ad8weqjx1ynrh7s8k25""",manager,35945535,10262493,2024-10-27 17:42:07,2
14805,"""01jb727x90ga4ncv4m705rh0rm""",client,36099337,0,2024-10-27 13:21:56,1
...,...,...,...,...,...,...
18252,"""01jb7sndn0b70srwkc6xjd15c0""",client,37557161,0,2024-10-27 20:11:16,1
18792,"""01jb7v0j8hkm3p8dagt0aae694""",manager,37557187,7996770,2024-10-27 20:34:49,1
18839,"""01jb7vgcc06pd71nvyag5ck7ma""",client,37557215,0,2024-10-27 20:43:28,1
18914,"""01jb7w5npw67ghth0ctgqkvrzr""",manager,37557215,6744792,2024-10-27 20:55:05,2


In [257]:
import pandas as pd

# Преобразование столбца 'created_at' в формат datetime
df = filter_ms.copy()
df['created_at'] = pd.to_datetime(df['created_at'])

# Пустой список для хранения результатов
results = []

# Группируем данные по 'entity_id'
for entity, group in df.groupby('entity_id'):
    # Разделение сообщений на клиентские и менеджерские
    client_message = group[group['type'] == 'client']
    manager_message = group[group['type'] == 'manager']

    if not client_message.empty and not manager_message.empty:
        # Сортируем сообщения по времени
        client_message = client_message.sort_values(by='created_at')
        manager_message = manager_message.sort_values(by='created_at')

        # Объединение данных клиента и менеджера по 'entity_id'
        merged = pd.merge(client_message[['entity_id', 'created_at']],
                          manager_message[['entity_id', 'created_at', 'created_by']],
                          on='entity_id',
                          suffixes=('_client', '_manager'))

        # Добавляем столбец идентификатора менеджера
        merged['mop_id'] = merged['created_by'].where(merged['created_by'] != 0, None).fillna(0).astype(int)

        # Временные параметры
        work_start_time = pd.to_datetime('09:30', format='%H:%M').time()
        work_end_time = pd.to_datetime('00:00', format='%H:%M').time()

        # Функция расчета времени ответа с учетом условий
        def calculate_response_time(row):
            client_time = row['created_at_client']
            manager_time = row['created_at_manager']

            # Условие 1: клиент написал в рабочее время, а менеджер ответил на следующий день
            if work_start_time <= client_time.time() <= work_end_time and manager_time.date() > client_time.date():
               time_to_midnight = (pd.Timestamp(client_time.date() + pd.Timedelta('1 day')) - client_time).total_seconds() / 60
               time_from_morning = (manager_time - pd.Timestamp(manager_time.date()) + pd.Timedelta(hours=9, minutes=30)).total_seconds() / 60
               return time_to_midnight + time_from_morning

            # Условие 2: клиент написал и менеджер ответил в нерабочее время
            if client_time.time() < work_start_time and manager_time.time() < work_start_time:
                return (manager_time - client_time).total_seconds() / 60

            # Условие 3: клиент написал в нерабочее время, менеджер ответил в рабочее
            if client_time.time() < work_start_time and work_start_time <= manager_time.time() <= work_end_time:
                adjusted_client_time = pd.Timestamp(manager_time.date()) + pd.Timedelta(hours=9, minutes=30)
                return (manager_time - adjusted_client_time).total_seconds() / 60

            # Условие 4: клиент и менеджер оба написали в рабочее время
            return (manager_time - client_time).total_seconds() / 60

        # Применяем функцию ко всем строкам
        merged['response_time_minutes'] = merged.apply(calculate_response_time, axis=1)
        merged['response_time_minutes'] = merged['response_time_minutes'].abs()

        # Добавляем результат в список
        results.append(merged[['entity_id', 'response_time_minutes', 'mop_id']])


# Объединяем результаты в один DataFrame
final_result = pd.concat(results)

# Результат
final_result


,entity_id,response_time_minutes,mop_id
0,35810509,1.650000,10262493
0,35945535,3.516667,10262493
0,36406841,4.433333,11490278
0,36487759,18.166667,11490270
0,36501589,3.316667,6645315
...,...,...,...
0,37557039,0.966667,6645315
0,37557101,11.550000,11448610
0,37557137,9.933333,10262513
0,37557149,28.516667,10465254


In [258]:
final_result['mop_id'] = final_result['mop_id'].astype(int)

In [259]:
final_result[final_result['entity_id']==35945535]

,entity_id,response_time_minutes,mop_id
0,35945535,3.516667,10262493


In [272]:
filter_ms[filter_ms['entity_id']==37551477]

,message_id,type,entity_id,created_by,created_at,group_number
5762,"""01jb4d24ng3fg9btk1ejxnhcd1""",client,37551477,0,2024-10-26 12:33:18,1
14520,"""01jb766k2gzqr5yhs61bswntf0""",manager,37551477,11448610,2024-10-27 14:31:07,2


In [261]:
manager

,mop_id,name_mop,rop_id
0,6645315,Гюнель и Илина,1
1,6744792,Юля и Наташа,1
2,10262505,Вика и Марго,1
3,10262513,Ира и Варя,1
4,10465254,Настя и Даша,1
5,11448610,Вика и Катя,1
6,11490270,Аня и Ксюша,1
7,11490278,Ксюша и Джамиля,1
8,6780177,Даша и Даша,2
9,7436430,Софа и Алина,2


In [265]:
result = pd.merge(final_result, manager, how='left', on='mop_id')
result = result.dropna()
result

,entity_id,response_time_minutes,mop_id,name_mop,rop_id
0,35810509,1.650000,10262493,Ками и Мила,3
1,35945535,3.516667,10262493,Ками и Мила,3
2,36406841,4.433333,11490278,Ксюша и Джамиля,1
3,36487759,18.166667,11490270,Аня и Ксюша,1
4,36501589,3.316667,6645315,Гюнель и Илина,1
...,...,...,...,...,...
529,37557039,0.966667,6645315,Гюнель и Илина,1
530,37557101,11.550000,11448610,Вика и Катя,1
531,37557137,9.933333,10262513,Ира и Варя,1
532,37557149,28.516667,10465254,Настя и Даша,1


In [271]:
result[result['response_time_minutes']==1557.8166666666666]

,entity_id,response_time_minutes,mop_id,name_mop,rop_id
369,37551477,1557.816667,11448610,Вика и Катя,1


In [279]:
min_time = min(result.response_time_minutes)
max_time = max(result.response_time_minutes)
print('min time:', min_time, 'minutes')
print('max time:', max_time/60, 'hours')

min time: 0.21666666666666667 minutes
max time: 25.96361111111111 hours


In [276]:
avg_response_time = round(result.response_time_minutes.mean(), 3)
print(avg_response_time, 'minutes')

57.209 minutes


In [1]:
%clear